## Pandas indexing, inserting, etc. worksheet


In [1]:
import pandas as pd
import numpy as np

Dataframes to work with:

In [2]:
#Create dataframes:
df = pd.DataFrame(['A','B','placeholder','C','D'],index=[0,1,2,3,4])   #indices are for rows
print('df:\n', df)

df2 = pd.DataFrame(['E','F'],index=[0,1])                  
print('\ndf2:\n', df2)


df:
              0
0            A
1            B
2  placeholder
3            C
4            D

df2:
    0
0  E
1  F


In [3]:
#Notes on indexing:
print(df.loc[2])   #loc gets rows (or columns) with particular labels (returned by df.index[])
print(df.iloc[2])  #iloc gets rows (or columns at particular positions in the index (only integers))

0    placeholder
Name: 2, dtype: object
0    placeholder
Name: 2, dtype: object


### Inserting one dataframe into another

**Approach 1:** re-indexing

In [5]:
#Dataframe 1:
print('df:\n', df)

#Get the index of a specific value in a specific column:
ind = df.index[df[0]=='placeholder']                        #df.index gets the index, df[0] looks in column 0
print('placeholder index name:', ind)                       #the index is an index object, referring to the index lable, NOT a positional integer
loc = df.index.get_loc(ind[0])                              #get the integer position of the index specified
print('and position:', loc)                                 #the result is an integer

#Format second dataframe (to be inserted into first):
print('df2:\n', df2)                                        #dataframe 2
df2_oldind = df2.index.values                               #gets all the row index values for the df
print('df2 original indices:', df2_oldind)
df2_newind = pd.Series(np.arange(loc, loc+1, 1/len(df2)),name='i')   #create new indices that are in between the rows where the new dataframe will be inserted into df1
print('df2 desired new indices:', df2_newind)
df2 = pd.concat([df2,df2_newind],axis=1)                    #tack on new column with desired indices
print('df2 with new index column:\n', df2)
df2.set_index('i', inplace=True)                            #replace existing indices with new indices in added column
print('reindexed df2:\n', df2)

#Insert the second df into the placeholder location:
df.drop(ind, inplace=True)                                  #remove the placeholder row
print('original df with placeholder dropped:\n', df)
new = pd.concat([df,df2])                                   #tack second df onto end of first df
print('concatenated df:\n', new)
new.sort_index(axis=0, inplace=True)                        #sort new df by row indices
print('correctly sorted concatenated df:\n', new)


df:
              0
0            A
1            B
2  placeholder
3            C
4            D
placeholder index name: Int64Index([2], dtype='int64')
and position: 2
df2:
    0
0  E
1  F
df2 original indices: [0 1]
df2 desired new indices: 0    2.0
1    2.5
Name: i, dtype: float64
df2 with new index column:
    0    i
0  E  2.0
1  F  2.5
reindexed df2:
      0
i     
2.0  E
2.5  F
original df with placeholder dropped:
    0
0  A
1  B
3  C
4  D
concatenated df:
      0
0.0  A
1.0  B
3.0  C
4.0  D
2.0  E
2.5  F
correctly sorted concatenated df:
      0
0.0  A
1.0  B
2.0  E
2.5  F
3.0  C
4.0  D


**Approach 2:** splitting into two and appending, then rejoining. This is MUCH more efficient!

In [6]:
#Create dataframes:
df = pd.DataFrame(['A','B','placeholder','C','D'],index=[0,1,2,3,4])   #indices are for rows
print('df:\n', df)

df2 = pd.DataFrame(['E','F'],index=[0,1])                  
print('\ndf2:\n', df2)


df:
              0
0            A
1            B
2  placeholder
3            C
4            D

df2:
    0
0  E
1  F


In [7]:
#Get row index and location of placeholder:
ind = df.index[df[0]=='placeholder']                        #df.index gets the index, df[0] looks in column 0 (returns an index object not just the name)
loc = df.index.get_loc(ind[0])                              #get the integer position of the index specified (need to select first item (index name) in index object)

#Split original df into two, one for everything before the placeholder, one for everything after:
dfA = df[df.index < loc]                                    #create df for everything above placeholder
dfB = df[df.index > loc]                                    #create df for everything below placeholder
print('Split original df:\n', dfA, '\n', dfB, '\n')

#Append the three dfs to each other (part above, part to insert, part below):
new = dfA.append(df2, ignore_index=True)                    #append additional part to top part
new = new.append(dfB, ignore_index=True)                    #append bottom part to new df
print('New combined df:\n', new)

Split original df:
    0
0  A
1  B 
    0
3  C
4  D 

New combined df:
    0
0  A
1  B
2  E
3  F
4  C
5  D


### Concatenating mixed data into a single formatted line

In [8]:
#Create df to be formatted:
df = pd.DataFrame([['string',1,1.5], ['string',1,1.5], ['string',1,1.5]], 
                   index = [0,1,2], columns = ['A','B','C'])   
print('df:\n', df, '\n')

#Create df to take formatting from:
df2 = pd.DataFrame([';;-------------- ---------- ----------'], index=[0], columns=[0])
print('df2:\n', df2)

df:
         A  B    C
0  string  1  1.5
1  string  1  1.5
2  string  1  1.5 

df2:
                                         0
0  ;;-------------- ---------- ----------


In [11]:
#Create format string based on template file:
string = df2[0].loc[0]            #get string to model formatting on (the row of dashes in the template file)
cols = string.split()             #split string into chunks separated by whitespace
lengths = [len(item) for item in cols]                   #get length of each column
form = ''                                                #create empty string to fill
form = [form + '{:<' + str(l+1) + '}' for l in lengths]  #concatenate formatting info and return a list of format strings (one per column)
form = ''.join(form)                                     #join format strings into one for entire row

#Insert data
data = df.loc[0]                             #get line of data to be formatted
line = form.format(data[0],data[1],data[2])  #insert data into format string
print(line)


string           1          1.5        
